In [ ]:
# CHOISIR LE MODÈLE !!!
# Si on change de modèle, faut redémarrer la session colab / adapter son environnement
model_name = "unsloth/Qwen2.5-VL-7B-Instruct" # Compétition fermée
#model_name = "unsloth/Qwen3-VL-8B-Instruct" # Compétion ouverte, exemple de modèle (on peut monter à plus de 8B)
# https://colab.research.google.com/github/unslothai/notebooks/blob/main/nb/Qwen3_VL_(8B)-Vision.ipynb

# Je me suis fait manger mes crédits au moment où j'essayais de faire tourner Qwen3 (code ne change pas ensuite a priori)

In [2]:
%%capture
import os, re
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    import torch; v = re.match(r"[0-9]{1,}\.[0-9]{1,}", str(torch.__version__)).group(0)
    xformers = "xformers==" + ("0.0.33.post1" if v=="2.9" else "0.0.32.post2" if v=="2.8" else "0.0.29.post3")
    !pip install --no-deps bitsandbytes accelerate {xformers} peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets==4.3.0" "huggingface_hub>=0.34.0" hf_transfer
    !pip install --no-deps unsloth

if "2.5" and "7B" in model_name:
    print("Modèle 2.5, compétition fermée")
    !pip install transformers==4.56.2
elif "Qwen3" in model_name:
    print("Modèle 3, compétion ouverte (exemple de modèle)")
    !pip install transformers==4.57.1
else:
    print("Problème modèle")
!pip install --no-deps trl==0.22.2

In [4]:
from unsloth import FastVisionModel # FastLanguageModel for LLMs
import torch

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
fourbit_models = [ # On ne s'en sert pas ici
    "unsloth/Llama-3.2-11B-Vision-Instruct-bnb-4bit", # Llama 3.2 vision support
    "unsloth/Llama-3.2-11B-Vision-bnb-4bit",
    "unsloth/Llama-3.2-90B-Vision-Instruct-bnb-4bit", # Can fit in a 80GB card!
    "unsloth/Llama-3.2-90B-Vision-bnb-4bit",

    "unsloth/Pixtral-12B-2409-bnb-4bit",              # Pixtral fits in 16GB!
    "unsloth/Pixtral-12B-Base-2409-bnb-4bit",         # Pixtral base model

    "unsloth/Qwen2-VL-2B-Instruct-bnb-4bit",          # Qwen2 VL support
    "unsloth/Qwen2-VL-7B-Instruct-bnb-4bit",
    "unsloth/Qwen2-VL-72B-Instruct-bnb-4bit",

    "unsloth/llava-v1.6-mistral-7b-hf-bnb-4bit",      # Any Llava variant works!
    "unsloth/llava-1.5-7b-hf-bnb-4bit",
] # More models at https://huggingface.co/unsloth

model, tokenizer = FastVisionModel.from_pretrained(
    model_name,
    load_in_4bit = True, # Use 4bit to reduce memory use. False for 16bit LoRA.
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for long context
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2026.1.4: Fast Qwen2_5_Vl patching. Transformers: 4.56.2.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/6.90G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/791 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

...

added_tokens.json:   0%|          | 0.00/605 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

video_preprocessor_config.json:   0%|          | 0.00/935 [00:00<?, ?B/s]

chat_template.json: 0.00B [00:00, ?B/s]

In [5]:
model = FastVisionModel.get_peft_model(
    model,
    finetune_vision_layers     = True, # False if not finetuning vision layers
    finetune_language_layers   = True, # False if not finetuning language layers
    finetune_attention_modules = True, # False if not finetuning attention layers
    finetune_mlp_modules       = True, # False if not finetuning MLP layers

    r = 16,           # The larger, the higher the accuracy, but might overfit
    lora_alpha = 16,  # Recommended alpha == r at least
    lora_dropout = 0,
    bias = "none",
    random_state = 3407, # Changer ça même si ça change rien
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
    # target_modules = "all-linear", # Optional now! Can specify a list if needed
)

In [6]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [7]:
import json
import os

base_path = '/content/drive/MyDrive/EVAHAN/train_data/'

##dataset_files = ['Dataset_A.json', 'Dataset_B.json', 'Dataset_C.json']
dataset_files = ['Dataset_A.json', 'Dataset_C.json']
#dataset_files = ['Dataset_B.json']
combined_dataset = []

for filename in dataset_files:
    filepath = os.path.join(base_path, filename)
    with open(filepath, 'r') as f:
        data = json.load(f)
        combined_dataset.extend(data)

print(f"Successfully loaded and combined {len(dataset_files)} datasets.")
print(f"Total items in combined dataset: {len(combined_dataset)}")

Successfully loaded and combined 2 datasets.
Total items in combined dataset: 10000


In [14]:
from sklearn.model_selection import train_test_split


# ON SE RESTREINT À PEU DE % DU JEU DE DONNÉES pour tester des trucs
subset_size = 0.020
subset_data, _ = train_test_split(combined_dataset, test_size=1 - subset_size, random_state=42)

print(f"Using a {subset_size*100}% subset of the data: {len(subset_data)} samples")

train_data, temp_data = train_test_split(subset_data, test_size=0.25, random_state=42)

test_data, val_data = train_test_split(temp_data, test_size=0.4, random_state=42)

print(f"Training set size: {len(train_data)}")
print(f"Testing set size: {len(test_data)}")
print(f"Validation set size: {len(val_data)}")

Using a 2.0% subset of the data: 200 samples
Training set size: 150
Testing set size: 30
Validation set size: 20


In [15]:
import PIL.Image
print("PIL.Image imported.")

PIL.Image imported.


In [16]:
from tqdm import tqdm # héhé
import json #
import os
import cv2 # binarisation image?
import numpy as np

instruction = """Analyze the provided image of ancient Chinese texts.

**Task Guidelines:**
1. **Transcription:** Transcribe the characters into standard Traditional Chinese (Unicode). Do not modernize the grammar.
2. **Legibility:** - If a character is an archaic variant, use the standard Traditional Chinese equivalent.
    - If a character is completely illegible due to damage, use '□'.
3. **Output:** Return ONLY the JSON object below that can be loaded using the loads function.


{
  "transcription": "TEXT_HERE",
  "notes": "Brief notes on damage/layout"
}
"""
# 4. **Reading Order:** If there is text, the text is generally written vertically (top to bottom) and arranged in columns from right to left. Respect this strict reading order.

base_path = '/content/drive/MyDrive/EVAHAN/train_data/'


# Adapted convert_to_conversation function
def convert_to_conversation_new(sample):
    if "text" not in sample:
        return None

    image_path = os.path.join(base_path, sample["image_path"])
    # PEUT-ÊTRE MODIFIER ÇA ?
    try:
    #    # Original:
      image = PIL.Image.open(image_path).convert("RGB")
    #    # NEW: Load image using OpenCV
    #    image_cv2 = cv2.imread(image_path)
    #    if image_cv2 is None:
    #        print(f"Error: Could not load image {image_path}")
    #        return None

    #    # Convert to grayscale
    #    gray_image = cv2.cvtColor(image_cv2, cv2.COLOR_BGR2GRAY)

        # Apply adaptive thresholding (binarization) to remove shadows
        # ADAPTIVE_THRESH_GAUSSIAN_C: uses a gaussian weighted sum of neighborhood values
        # THRESH_BINARY: the type of thresholding applied
        # 255: max value to use with THRESH_BINARY
        # 11: block size (size of neighborhood to calculate threshold for)
        # 2: constant subtracted from the mean or weighted mean
    #    binarized_image = cv2.adaptiveThreshold(gray_image, 255,
    #                                             cv2.ADAPTIVE_THRESH_GAUSSIAN_C,
    #                                             cv2.THRESH_BINARY, 11, 2)

        # Convert OpenCV image (numpy array) back to PIL Image
     #   image = PIL.Image.fromarray(binarized_image).convert("RGB") # Ensure RGB for model input

    except Exception as e:
        print(f"Error processing image {image_path}: {e}")
        return None

    assistant_response_dict = {
        "transcription": sample["text"],
        "notes": ""
    }
    assistant_response_json_string = json.dumps(assistant_response_dict, ensure_ascii=False) # ensure_ascii=False to preserve Chinese characters

    conversation = [
        { "role": "user",
          "content" : [
            {"type" : "text",  "text"  : instruction},
            {"type" : "image", "image" : image} ]
        },
        { "role" : "assistant",
          "content" : [
            {"type" : "text",  "text"  : assistant_response_json_string} ]
        },
    ]
    return { "messages" : conversation }

converted_train_dataset = [convert_to_conversation_new(sample) for sample in tqdm(train_data) if convert_to_conversation_new(sample) is not None]
converted_val_dataset = [convert_to_conversation_new(sample) for sample in tqdm(val_data) if convert_to_conversation_new(sample) is not None]

print(f"Converted training dataset size: {len(converted_train_dataset)}")
print(f"Converted validation dataset size: {len(converted_val_dataset)}")

100%|██████████| 20/20 [00:09<00:00,  2.06it/s]

Converted training dataset size: 150
Converted validation dataset size: 20


In [17]:
train_data[0]

{'image_path': 'Dataset_A/a_0065.jpg', 'text': '五采覆壙上群鶴翔雲中觀者歎異之公純誠精粹負氣正大雖'}

In [18]:
converted_train_dataset[0]

{'messages': [{'role': 'user',
   'content': [{'type': 'text',
     'text': 'Analyze the provided image of ancient Chinese texts.\n\n**Task Guidelines:**\n1. **Transcription:** Transcribe the characters into standard Traditional Chinese (Unicode). Do not modernize the grammar.\n2. **Legibility:** - If a character is an archaic variant, use the standard Traditional Chinese equivalent.\n    - If a character is completely illegible due to damage, use \'□\'.\n3. **Output:** Return ONLY the JSON object below that can be loaded using the loads function.\n\n\n{\n  "transcription": "TEXT_HERE",\n  "notes": "Brief notes on damage/layout"\n}\n'},
    {'type': 'image',
     'image': <PIL.Image.Image image mode=RGB size=4425x204>}]},
  {'role': 'assistant',
   'content': [{'type': 'text',
     'text': '{"transcription": "五采覆壙上群鶴翔雲中觀者歎異之公純誠精粹負氣正大雖", "notes": ""}'}]}]}

In [19]:
from unsloth.trainer import UnslothVisionDataCollator
from trl import SFTTrainer, SFTConfig

FastVisionModel.for_training(model) # Enable for training!

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    data_collator = UnslothVisionDataCollator(model, tokenizer), # Must use!
    train_dataset = converted_train_dataset, # Updated to use the new training dataset
    eval_dataset = converted_val_dataset,    # Added for evaluation
    args = SFTConfig(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 5,
        # Set num_train_epochs to 1 and max_steps to -1 to train for one full epoch.
        # Adjust these values based on desired training duration and dataset size.
        max_steps = -1,                    # Set to -1 to train for num_train_epochs
        num_train_epochs = 1,              # Train for 1 full epoch
        learning_rate = 2e-4,
        logging_steps = 1,
        optim = "adamw_8bit",
        weight_decay = 0.001,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "outputs",
        report_to = "none",     # For Weights and Biases

        # You MUST put the below items for vision finetuning:
        remove_unused_columns = False,
        dataset_text_field = "",
        dataset_kwargs = {"skip_prepare_dataset": True},
        max_length = 2048,
    ),
)

Unsloth: Model does not have a default image size - using 512


In [20]:
# @title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = Tesla T4. Max memory = 14.741 GB.
6.822 GB of memory reserved.


In [21]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 150 | Num Epochs = 1 | Total steps = 19
O^O/ \_/ \    Batch size per device = 2 | Gradient accumulation steps = 4
\        /    Data Parallel GPUs = 1 | Total batch size (2 x 4 x 1) = 8
 "-____-"     Trainable parameters = 51,521,536 of 8,343,688,192 (0.62% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss
1,4.599300
2,4.579200
3,4.389300
4,3.963400


KeyboardInterrupt: 

In [ ]:
# @title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory / max_memory * 100, 3)
lora_percentage = round(used_memory_for_lora / max_memory * 100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(
    f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training."
)
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

In [23]:
import os
import PIL.Image

FastVisionModel.for_inference(model) # Enable for inference!

test_example = test_data[0]

print(test_example['text'])

image_path = os.path.join(base_path, test_example["image_path"])
image = PIL.Image.open(image_path).convert("RGB")

#instruction_ = instruction

messages = [
    {"role": "user", "content": [
        {"type": "image"},
        {"type": "text", "text": instruction}
    ]}
]
input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)
inputs = tokenizer(
    image,
    input_text,
    add_special_tokens = False,
    return_tensors = "pt",
).to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer, skip_prompt = True)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128,
                   use_cache = False, temperature = 1.5, min_p = 0.1) # Changed use_cache to False

诣
```json
{
  "transcription": "海",
  "notes": ""
}
```<|im_end|>


In [24]:
import os

!wget -O task_a_c_eva.py https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_a_c_eva.py
!wget -O task_b_eva.py https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_b_eva.py

print("Downloaded task_a_c_eva.py and task_b_eva.py")

--2026-02-03 23:38:27--  https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_a_c_eva.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14094 (14K) [text/plain]
Saving to: ‘task_a_c_eva.py’

task_a_c_eva.py     100%[===================>]  13.76K  --.-KB/s    in 0s      

2026-02-03 23:38:27 (106 MB/s) - ‘task_a_c_eva.py’ saved [14094/14094]

--2026-02-03 23:38:28--  https://raw.githubusercontent.com/GoThereGit/EvaHan/refs/heads/main/task_b_eva.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 20275 (20K) [tex

In [ ]:
FastVisionModel.for_inference(model) # Ensure model is in inference mode

import json
import re
from tqdm import tqdm
import os
import PIL.Image
import sys

if "/content/" not in sys.path:
    sys.path.append("/content/")

if 'task_a_c_eva' in sys.modules:
    del sys.modules['task_a_c_eva']
if 'task_b_eva' in sys.modules:
    del sys.modules['task_b_eva']

from task_a_c_eva import calculate_char_metrics
# from task_b_eva import LayoutEvaluator PETITES MODIF A FAIRE LÀ

sum_score = 0
idx = 0

for i, test_example in tqdm(enumerate(test_data[:20]), total=20, desc="Performing inference and evaluation"):
    image_path = os.path.join(base_path, test_example["image_path"])
    try:
        image = PIL.Image.open(image_path).convert("RGB")
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")
        continue

    messages = [
        {"role": "user", "content": [
            {"type": "image"},
            {"type": "text", "text": instruction}
        ]}
    ]

    input_text = tokenizer.apply_chat_template(messages, add_generation_prompt = True)

    inputs = tokenizer(
        image,
        input_text,
        add_special_tokens = False,
        return_tensors = "pt",
    ).to("cuda")

    outputs = model.generate(**inputs, max_new_tokens = 128,
                        use_cache = False, temperature = 1.5, min_p = 0.1)

    generated_text_tokens = outputs[0][len(inputs["input_ids"][0]):]
    model_output_raw = tokenizer.decode(generated_text_tokens, skip_special_tokens=True)

    # --- DÉBUT DES MODIFICATIONS DE NETTOYAGE ---

    # 1. Nettoyage basique des tokens de fin
    model_output_cleaned = model_output_raw.replace("<|im_end|>", "").strip()

    # 2. Nettoyage des balises Markdown (```json ... ```)
    # On retire ```json au début et ``` à la fin, peu importe les espaces
    model_output_cleaned = re.sub(r'^```json\s*', '', model_output_cleaned, flags=re.MULTILINE)
    model_output_cleaned = re.sub(r'\s*```$', '', model_output_cleaned, flags=re.MULTILINE)
    model_output_cleaned = model_output_cleaned.strip()

    predicted_transcription = "N/A (parsing error)"

    try:
        # On prépare pour le JSON (ta méthode existante pour les sauts de ligne)
        model_output_for_json = model_output_cleaned.replace('\n', '\\n')

        parsed_output = json.loads(model_output_for_json)
        predicted_transcription = parsed_output.get("transcription", "N/A (transcription key missing)")

    except json.JSONDecodeError as e:
        print(f"JSON decoding error for output: {model_output_cleaned} - {e}")
        # IMPORTANT : En cas d'erreur, on met une chaîne vide pour le calcul du score.
        # Si on met le message d'erreur "JSON Error...", cela fausse le calcul CER (insertions massives).
        predicted_transcription = ""

    # --- FIN DES MODIFICATIONS ---

    ground_truth_transcription = test_example.get("text", "N/A (ground truth missing)")

    print(f"\n--- Example {i+1} ---")
    print(f"Model Output:     {predicted_transcription}")
    print(f"Ground Truth:     {ground_truth_transcription}")

    dataset_name = image_path.split('/')[-2]
    if "Dataset_B" in dataset_name:

        print("Evaluation (Dataset B): Not applicable. Model outputs transcription, but ground truth is for layout detection.")
        # RÉADAPTER POUR DATASET B

    else:

        if ground_truth_transcription != "N/A (ground truth missing)":

            # Note : Si predicted_transcription est vide (suite erreur JSON), le score sera bas mais pas délirant (-54)
            metrics_ac = calculate_char_metrics(ground_truth_transcription, predicted_transcription)

            score_ac = metrics_ac.get("comprehensive_score", 0) # Valeur par défaut 0 si manquant
            score_cer = metrics_ac.get("cer", "N/A")
            score_precision = metrics_ac.get("precision", "N/A")
            score_recall = metrics_ac.get("recall", "N/A")
            score_f1 = metrics_ac.get("f1", "N/A")
            score_ned = metrics_ac.get("ned", "N/A")

            print(f"Evaluation (Dataset A/C): Comprehensive Score = {score_ac}")
            print(f"Precisely: CER={score_cer}, P={score_precision}, R={score_recall}, F1={score_f1}, ned={score_ned}")

            idx += 1
            sum_score += score_ac # J'ai renommé 'sum' en 'sum_score' car 'sum' est un mot clé python
        else:
            print("Evaluation (Dataset A/C): Skipping due to missing ground truth.")


if idx > 0:
    print(f"Mean: {sum_score / idx}")
else:
    print("Mean: N/A (No valid examples processed)")

Performing inference and evaluation:   5%|▌         | 1/20 [00:11<03:39, 11.55s/it]

JSON decoding error for output: {
  "transcription": "海",
  "notes": "No visible damage or layout issues detected."
} - Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

--- Example 1 ---
Model Output:     
Ground Truth:     诣
Evaluation (Dataset A/C): Comprehensive Score = 0.0
Precisely: CER=1.0, P=0.0, R=0.0, F1=0.0, ned=1.0


In [ ]:
model.save_pretrained("lora_model_QWEN2.5_EVAHAN")
tokenizer.save_pretrained("lora_model_QWEN2.5_EVAHAN")

In [ ]:
print(test_data[9]['text']) # si + de 10 exemples, ne pas oublier de changer le 9 !
print(json.loads(model_output_cleaned)['transcription'])
